# Random Effect Logistic Regression by MLMC Variational Bayes

## Model Description

##### Random Effect Models:
For $n=1,2,...$,
<br>&nbsp;&nbsp;&nbsp;&nbsp;
$Z_n \sim N(0,\tau^2)$
<br>&nbsp;&nbsp;&nbsp;&nbsp;
$Y_{n,t} \sim \text{Bernoulli}\left(\frac{1}{1+\exp(- Z_n - \beta_0 - \beta^T x_{n,t})}\right)$
<br>
for $t=1, ..., T$. This model carries out dimentionality reduction of binary observations $y_{n,k}$'s. Here, the dimention of $\beta$ and $x_{n,t}$ is $D$.<br>
As variational approximation of the posterior $p(z_n|y_n)$, we use $q(z_n)= N(z_n;\mu_n, \sigma_n^2)$.

To constrain the parameter $\tau^2>0$, we parametrize $\tau^2$ as $\tau^2 = \mathrm{softplus}(\alpha)$, where $\mathrm{softplus}(x) := \log(1+\exp(x))>0$.

##### Possible Extension:

By adding $\bar x_n=\frac{1}{T}\sum_t x_{n,t}$ to the predictors as 
<br>&nbsp;&nbsp;&nbsp;&nbsp;
$Y_{n,t} \sim \text{Bernoulli}\left(\frac{1}{1+\exp(- Z_n - \beta_0 - \beta^T x_{n,t}- \gamma\bar x_n)}\right)$,
<br>
we can obtain correlated random effect models.

#### We do not consider the use of Renyi divergences

## Import Packages

In [1]:
# Turn GPUs off
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
from scipy.stats import bernoulli, norm
from matplotlib import pyplot as plt
import time
import datetime
np.random.seed(0)

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
import sys
sys.path.append('../')
from src.models.random_effect_logistic_regression import random_effect_logistic_regression as RELR

## Utilities

In [5]:
sigmoid = lambda x: 1/(1+np.exp(-x))
softplus = lambda x: np.log(1+np.exp(x))

def timestamp():
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d%H%M%S")  

## Generate Toy Dataset

In [6]:
N = 1000
D = 3
T = 2

In [7]:
# Paramters
alpha = np.float64(1.)
beta0 = np.float64(0.)
beta  = np.array([0.25, 0.50, 0.75]) #np.random.randn(D) / np.sqrt(D)
param0 = {
    'alpha': alpha,
    'beta0': beta0,
    'beta': beta
}

In [8]:
# We assume that we have infinite amount of data.
# Thus, generator of the data is implemented.
def generate_data(N, D, T, beta0, beta, alpha):
    """
    Genarate N samples of data from the model with parameter [beta0, beta, alpha]. 
    Returns:
    x: 3-d array of size [N, T, D]
    y: 2-d array of size [N, T]
    z: 1-d array of size [n_MC, N]
    """

    z = np.random.randn(N) * softplus(alpha)**(1/2.)
    x = np.random.randn(N*T*D).reshape([N,T,D])
    y = bernoulli(p=sigmoid(beta0+x@beta+z.reshape([N,1]))).rvs()
    return x,y,z

To constrain the parameter $\tau^2>0$, we parametrize $\tau^2$ as $\tau^2 = \mathrm{softplus}(\alpha)$, where $\mathrm{softplus}(x) := \log(1+\exp(x))>0$.

## Sigmoid Normal Integral Approximation of Evidence

Ref: Barber Bishop(1998), PRML(2006)

Sigmoid normal integral approximation is:
$$\int \sigma(z + \mu)N(z;0,\tau^2)\mathrm{d}z 
\approx \sigma(\kappa \mu) \ \ \ 
\text{where}\ \ \ 
\kappa = \left(\frac{1}{1 + \pi\tau^2/8}\right)^{\frac{1}{2}}$$
This is applied to the probability of y: 
$$Pr(Y=1|X=x) = \int \sigma(z + \beta_0 + \beta^T x)N(z;0,\tau^2)\mathrm{d}z,$$
where $\tau^2 = \mathrm{softmax}(\alpha)$.

## Laplace Approximation of Posterior

Laplace approximation of $p(z|x,y)$ is:
$$p(z|x,y) \approx \exp\left[\frac{1}{2}\left(\frac{\mathrm{d}^2}{\mathrm{d}z^2}\log p(\hat z|x,y) \right)(z - \hat z)^2 + \text{const.}\right] = N\left(z;\hat z, \left(-\frac{\mathrm{d}^2}{\mathrm{d}z^2}\log p(\hat z|x,y) \right)^{-1}\right),$$
where $\hat z$ is the maximum a postriori (MAP), so that $\hat z = \arg\max_{z}\log p(z|x,y) = \arg\max_{z}\log p(x,y|z)p(z)$

## IW-ELBO approximation of Evidence

Importance weighted evidence lower bound (IW-ELBO) is an approximation of the evidence, and its Monte Carlo estimator is defined as:
$$\mathrm{IW}\text{-}\mathrm{ELBO}(x,z_{1:K}):=\frac{1}{N}\sum_{n=1}^{N}\log \frac{1}{K}\sum_{k=1}^{K}\left(\frac{p(x_n,z_{nk})}{q(z_{nk};x_n)}\right),$$
where $z_{nk}$ is sampled as $z_{nk}\sim q(z;x_n)$.

## MLMC

## SUMO

#### Variance Analysis of SUMO:
We assume that $\mathbb{P}(k\leq\mathcal{K})=\frac{1}{k}$ for $\mathcal{K}\leq K_{\text{max}}$, and accordingly, $\mathbb{P}(k=\mathcal{K}<K_{\text{max}})=\frac{1}{k(k+1)}$. Also, we know convergence rate of $\Delta_k:=\mathrm{IWELBO(x,z_{1:k})} - \mathrm{IWELBO(x,z_{1:k-1})}$ is as following:

- $\mathrm{E}|\Delta_k| = \mathcal{O}(1/k)$
- $\mathrm{E}|\Delta_k|^2 = \mathcal{O}(1/k^2) \geq \mathrm{Var}[\Delta_k]$
- $\mathrm{E}[\Delta_k] = \mathcal{O}(1/k^2)$　　　←（∵$\sum_{k=K+1}^{\infty}\mathrm{E}[\Delta_k] = \mathcal{O}(\frac{1}{K})$ and $0\leq\mathrm{E}[\Delta_k]$ for all $k$.）

Using above properties, we can analyze the order of magnitude of the variance of SUMO as:
\begin{align}
\mathrm{Var}[SUMO_{\leq K_{\text{max}}}]
&= \mathrm{Var}_{\mathcal{K}, \Delta_{1:K_{\text{max}}}} 
\left( \sum_{k=1}^{K_{\text{max}}} \frac { \mathbb{1}_{(k\leq\mathcal{K})} } 
{ \mathbb{P}(k\leq\mathcal{K}) } \Delta_k \right) \\
&= \mathrm{E}_{\mathcal{K}}\mathrm{Var}_{\Delta_{1:K_{\text{max}}}}
\left( \sum_{k=1}^{\mathcal{K}} k \Delta_k | \mathcal {K}\right)
 + \mathrm{Var}_{\mathcal{K}}\mathrm{E}_{\Delta_{1:K_{\text{max}}}}
\left( \sum_{k=1}^{\mathcal{K}} k \Delta_k | \mathcal {K}\right) \\
&= \mathrm{E}_{\mathcal{K}}\sum_{k=1}^{\mathcal{K}} k^2 \mathrm{Var}_{\Delta_{1:K_{\text{max}}}}
\left( \Delta_k | \mathcal {K}\right)
 + \mathrm{Var}_{\mathcal{K}} \sum_{k=1}^{\mathcal{K}} k \mathrm{E}_{\Delta_{1:K_{\text{max}}}}
\left( \Delta_k | \mathcal {K}\right) \\
&= \mathrm{E}_{\mathcal{K}}\sum_{k=1}^{\mathcal{K}} k^2 \mathcal{O}(1/k^2)
 + \mathrm{Var}_{\mathcal{K}} \sum_{k=1}^{\mathcal{K}} k \mathcal{O}(1/k^2) \\
&= \mathrm{E}_{\mathcal{K}} \mathcal{O}(\mathcal{K})
 + \mathrm{Var}_{\mathcal{K}}\mathcal{O}(\log \mathcal{K}) \\
&= \mathcal{O}(\log K_{\text{max}}) + \text{const.}
\end{align}

Note that the expected complexity of SUMO is $\mathrm{E}[\mathcal{\mathcal{K}}] = \mathcal{O}(\log K_{\text{max}})$. Therefore, the "variance per reciprocal complexity" for SUMO is $\mathcal{O}\left((\log K_{\text{max}})^2\right)$. Those for NMC and MLMC are $\mathcal{O}(K_{\text{max}})$ and $\mathcal{O}(1)$ respectively.

## Make sure that implementations are consistent

In [9]:
x,y,_ = generate_data(N=10000, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
model = RELR(alpha, beta0, beta)
q_param = model.laplace_approx(x, y)    

In [10]:
q_param

{'mu': array([ 0.85936059,  0.09936386, -0.00549784, ...,  0.67903061,
         0.65668019,  0.84328928]),
 'sigma': array([0.93545707, 0.9365725 , 0.89478043, ..., 0.94620007, 0.95501274,
        0.91546504])}

In [11]:
model.sigmoid_normal_likelihood(x,y)

<tf.Tensor: shape=(), dtype=float64, numpy=-1.269810829237006>

In [12]:
signorm_likelihood = sigmoid_normal_likelihood(x, y, beta0, beta, alpha).numpy()
elbo_likelihood = IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=1).numpy()
iwelbo_likelihood = IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=64).numpy()
iwelbo_likelihood_mlmc = IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=6, w0=0.9, randomize=False).numpy()
iwelbo_likelihood_randmlmc = IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=6, w0=0.9, randomize=True).numpy()
iwelbo_likelihood_sumo = IWELBO_SUMO(x, y, beta0, beta, alpha, mu, sigma, K_max=10000).numpy()

NameError: name 'sigmoid_normal_likelihood' is not defined

In [ ]:
signorm_likelihood, elbo_likelihood, iwelbo_likelihood

In [ ]:
iwelbo_likelihood_mlmc, iwelbo_likelihood_randmlmc, iwelbo_likelihood_sumo 

## MLMC codition check for objective

In [ ]:
def conv_stats_dIWELBO(x, y, beta0, beta, alpha, mu, sigma, level=1):
    # Compute dIWELBO (and IWELBO) for each sample and 
    # summarize them into several statistics.

    N, = mu.shape
    n_MC = 2**level
    z = norm(loc=mu, scale=sigma).rvs([n_MC, N])
    
    diwelbos = pointwise_dIWELBO(x, y, z, beta0, beta, alpha, mu, sigma).numpy()
    iwelbos = pointwise_IWELBO(x, y, z, beta0, beta, alpha, mu, sigma).numpy()
    
    return {'mean_dIWELBO':np.mean(diwelbos), 
            'mean_abs_dIWELBO':np.mean(np.abs(diwelbos)), 
            'mean_squared_dIWELBO':np.mean(diwelbos**2),
            'var_dIWELBO':np.var(diwelbos), 
            'var_IWELBO':np.var(iwelbos)}

In [ ]:
def tmp(l):
    N0 = 2000000
    x,y,_ = generate_data(N=N0//2**l, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
    mu, sigma = laplace_approx(x, y, beta0, beta, alpha)
    return conv_stats_dIWELBO(x, y, beta0, beta, alpha, mu, sigma, level=l)
L=13
conv_stats = [tmp(l) for l in range(L)]
conv_stats = pd.DataFrame(conv_stats)

In [ ]:
# plot results
plt.plot(conv_stats[['mean_abs_dIWELBO', 'var_dIWELBO', 'var_IWELBO']])

# plot O(2^{-l/2}), O(2^{-l}), O(2^{-2l})
s,t = conv_stats[['mean_abs_dIWELBO', 'var_dIWELBO']].iloc[0]
plt.plot(t*2.**(-np.arange(L)/2), c='grey')
plt.plot(t*2.**(-np.arange(L)), c='grey')
plt.plot(t*2.**(-np.arange(L)*2), c='grey')

plt.legend([r'$\mathrm{E} | \Delta \mathrm{IW}$-$\mathrm{ELBO}|$', 
            r'$\mathrm{Var}[\Delta \mathrm{IW}$-$\mathrm{ELBO}]$', 
            r'$\mathrm{Var}[\mathrm{IW}$-$\mathrm{ELBO}]$',
            r'$O(2^{-\ell/2}), O(2^{-\ell}), O(2^{-2\ell})$'])
plt.xlabel('Level')
plt.yscale('log')

In [ ]:
# compute cost-variance efficiency of MLMC 
C = [1 if l==0 else 2**(l+1) for l in range(L)]
V = conv_stats['var_dIWELBO']
N = np.sqrt(V/C)
cost_mlmc = [np.sum([np.sqrt(c*v) for c,v in zip(C[:l+1], V[:l+1])])**2  for l in range(L)]
cost_nmc = [2**l * v for l,v in enumerate(conv_stats['var_IWELBO'])]

In [ ]:
plt.figure(figsize=[12,4])
# plot theoretical computational efficiency
plt.subplot(1,2,1)
plt.plot(cost_mlmc)
plt.plot(cost_nmc)
plt.yscale('log')
plt.ylabel('Variance per Reciprocal Complexity')
plt.xlabel('level')
plt.legend(['MLMC', 'Nested MC'])
# plot N_l's for MLMC
plt.subplot(1,2,2)
plt.plot(N)
plt.yscale('log')
plt.ylabel(r'$N_l$')
plt.xlabel('level')
print('(Theoretical) Variance per Reciprocal Complexity and N_l\'s for MLMC')

In [ ]:
conv_stats

## MLMC codition check for gradients

In [ ]:
def conv_stats_grad_dIWELBO(x, y, beta0, beta, alpha, mu, sigma, level=1):
    # Compute the gradient of dIWELBO (and IWELBO) for each sample and 
    # summarize them into several statistics.

    N, = mu.shape
    n_MC = 2**level
    z = norm(loc=mu, scale=sigma).rvs([n_MC, N]).T
    
    param = tf.concat([beta, [beta0], [alpha]], axis=0)
    param = tf.Variable(param, dtype=tf.float64)
    params = tf.reshape(param, [1,D+2]) * np.ones([N,1])

    mu, sigma = laplace_approx(x, y, beta0, beta, alpha)        
    
    # Define a gradient function to be vectorized (vectorization for better performance)
    def get_grad(args):
        # get gradient of dIWELBO (and IWELBO) given one sample
        param, x_, y_, z_, mu, sigma = args
        z_ = tf.reshape(z_, [-1,1])
        with tf.GradientTape(persistent=True) as g:
            g.watch(param)
            beta_ = param[0,:D]
            beta0_ = param[0,D]
            alpha_ = param[0,D+1]
            diwelbos = pointwise_dIWELBO(x_, y_, z_, beta0_, beta_, alpha_, mu, sigma)
            iwelbos = pointwise_IWELBO(x_, y_, z_, beta0_, beta_, alpha_, mu, sigma)    
        a = g.gradient(diwelbos, param)
        b = g.gradient(iwelbos, param)
        del g
        return a,b
    
    # Compute the gradient of dIWELBO (and IWELBO) for each sample
    args = [tf.expand_dims(arg, axis=1) for arg in [params, x, y, z, mu, sigma]]
    grads = tf.vectorized_map(get_grad, args)
    
    grad_diwelbos = tf.squeeze(grads[0])#[:D+1]
    grad_iwelbos = tf.squeeze(grads[1])#[:D+1]
    
    # return summary statistics of the gradients
    return {'norm_mean_grad_dIWELBO': np.linalg.norm(np.mean(grad_diwelbos, axis=0)), 
            'mean_norm_grad_dIWELBO': np.mean(np.linalg.norm(grad_diwelbos, axis=1)), 
            'mean_squared_norm_grad_dIWELBO': np.mean(np.linalg.norm(grad_diwelbos, axis=1)**2),
            'trace_covariance_grad_dIWELBO': np.sum(np.var(grad_diwelbos, axis=0)), 
            'trace_covariance_grad_IWELBO': np.sum(np.var(grad_iwelbos, axis=0))}

In [ ]:
def tmp(l):
    N0 = 2000000
    x,y,_ = generate_data(N=N0//2**l, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
    mu, sigma = laplace_approx(x, y, beta0, beta, alpha)
    return conv_stats_grad_dIWELBO(x, y, beta0, beta, alpha, mu, sigma, level=l)
L=13
conv_stats = [tmp(l) for l in range(L)]
conv_stats = pd.DataFrame(conv_stats)

In [ ]:
# plot results
plt.plot(conv_stats[['norm_mean_grad_dIWELBO', 'trace_covariance_grad_dIWELBO', 'trace_covariance_grad_IWELBO']])

# plot O(2^{-l/2}), O(2^{-l}), O(2^{-2l})
s,t = conv_stats[['norm_mean_grad_dIWELBO', 'trace_covariance_grad_dIWELBO']].iloc[0]
plt.plot(t*2.**(-np.arange(L)/2), c='grey')
plt.plot(t*2.**(-np.arange(L)), c='grey')
plt.plot(t*2.**(-np.arange(L)*2), c='grey')

plt.legend([r'$||\mathrm{E} [\nabla (\Delta \mathrm{IW}$-$\mathrm{ELBO})]||_2$', 
            r'$\mathrm{tr}(\mathrm{Cov}[\nabla(\Delta \mathrm{IW}$-$\mathrm{ELBO})])$', 
            r'$\mathrm{tr}(\mathrm{Cov}[\nabla(\mathrm{IW}$-$\mathrm{ELBO})])$',
            r'$O(2^{-\ell/2}), O(2^{-\ell}), O(2^{-2\ell})$'])
plt.xlabel('Level')
plt.yscale('log')

In [ ]:
# compute cost-variance efficiency of MLMC 
C = [1 if l==0 else 2**(l+1) for l in range(L)]
V = conv_stats['trace_covariance_grad_dIWELBO']
N = np.sqrt(V/C)
cost_mlmc = [np.sum([np.sqrt(c*v) for c,v in zip(C[:l+1], V[:l+1])])**2  for l in range(L)]
cost_nmc = [2**l * v for l,v in enumerate(conv_stats['trace_covariance_grad_IWELBO'])]

In [ ]:
plt.figure(figsize=[12,4])
# plot theoretical computational efficiency
plt.subplot(1,2,1)
plt.plot(cost_mlmc)
plt.plot(cost_nmc)
plt.yscale('log')
plt.ylabel('Variance per Reciprocal Complexity')
plt.xlabel('level')
plt.legend(['MLMC', 'Nested MC'])
# plot N_l's for MLMC
plt.subplot(1,2,2)
plt.plot(N)
plt.yscale('log')
plt.ylabel(r'$N_l$')
plt.xlabel('level')
print('(Theoretical) Variance per Reciprocal Complexity and N_l\'s for MLMC')

In [ ]:
conv_stats

In [ ]:
def conv_stats_dSUMO(x, y, beta0, beta, alpha, mu, sigma, level=1):
    # Compute dIWELBO (and IWELBO) for each sample and 
    # summarize them into several statistics.

    N, = mu.shape
    n_MC = 2**level
    z = norm(loc=mu, scale=sigma).rvs([n_MC, N])
    
    dsumos = 0
    dsumos += pointwise_IWELBO(x, y, z[ :,:], beta0, beta, alpha, mu, sigma).numpy()
    dsumos -= pointwise_IWELBO(x, y, z[1:,:], beta0, beta, alpha, mu, sigma).numpy()
    
    dN = N//10
    var_sumos = dN * np.var([
        IWELBO_SUMO(
            x[dN*i:dN*(i+1)], y[dN*i:dN*(i+1)], beta0, beta, alpha,
            mu[dN*i:dN*(i+1)], sigma[dN*i:dN*(i+1)], K_max=2**level
            ) 
        for i in range(10)])
    
    return {'mean_dSUMO':np.mean(dsumos), 
            'mean_abs_dSUMO':np.mean(np.abs(dsumos)), 
            'mean_squared_dSUMO':np.mean(dsumos**2),
            'var_dSUMO':np.var(dsumos),
            'var_SUMO':var_sumos}

In [ ]:
def tmp(l):
    N0 = 2000000
    x,y,_ = generate_data(N=N0//2**l, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
    mu, sigma = laplace_approx(x, y, beta0, beta, alpha)
    return conv_stats_dSUMO(x, y, beta0, beta, alpha, mu, sigma, level=l)
L=13
conv_stats = [tmp(l) for l in range(L)]
conv_stats = pd.DataFrame(conv_stats)

In [ ]:
# plot results
plt.plot(conv_stats[['mean_abs_dSUMO', 'var_dSUMO', 'var_SUMO']])

# plot O(2^{-l/2}), O(2^{-l}), O(2^{-2l})
s,t = conv_stats[['mean_abs_dSUMO', 'var_dSUMO']].iloc[1]
plt.plot(s*2.**(-np.arange(L)), c='grey')
plt.plot(t*2.**(-np.arange(L)*2), c='grey')

plt.legend([r'$\mathrm{E}  |\Delta^{\mathrm{SUMO}}_{2^{\ell}}|$', 
            r'$\mathrm{Var}[\Delta^{\mathrm{SUMO}}_{2^{\ell}}]$', 
            r'$\mathrm{Var}[\mathrm{SUMO}_{2^{\ell}}]$', 
            r'$O(2^{-\ell}), O(2^{-2\ell})$'])
plt.xlabel('Level')
plt.yscale('log')

In [ ]:
conv_stats

## Cost comparison of objective

In [ ]:
L = 13
objectives = {
    'NMC':      lambda x,y,mu,sigma,level: IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=2**level),
    'MLMC':     lambda x,y,mu,sigma,level: IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=level, w0=0.95, b=1.8, randomize=False),
    'RandMLMC': lambda x,y,mu,sigma,level: IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=level, w0=0.95, b=1.8, randomize=True),
    'SUMO':     lambda x,y,mu,sigma,level: IWELBO_SUMO(x, y, beta0, beta, alpha, mu, sigma, K_max=2**level)
}
results = {'NMC':[], 'MLMC':[], 'RandMLMC':[], 'SUMO':[]}
runtime = {'NMC':[], 'MLMC':[], 'RandMLMC':[], 'SUMO':[]}

for name, obj in objectives.items():
    
    # evaluate variance
    for i in range(100):
        results[name].append([])
        x,y,_ = generate_data(N=4000, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
        mu, sigma = laplace_approx(x, y, beta0, beta, alpha)
        for level in range(L):
            results[name][i].append( obj(x,y,mu,sigma,level).numpy() )
    
    # evaluate runtime
    x,y,_ = generate_data(N=20000, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
    mu, sigma = laplace_approx(x, y, beta0, beta, alpha)    
    for level in range(L):
        if level>10 and name=='NMC':
            start = time.time()
            obj(*[vec[:200] for vec in [x,y,mu,sigma]], level)
            end = time.time()
            runtime[name].append((end - start)*100)
        else:
            start = time.time()
            obj(x,y,mu,sigma,level)
            end = time.time()
            runtime[name].append(end - start)

In [ ]:
for ests, rtime in zip(results.values(), runtime.values()):
    var_per_recip_runtime = np.array(ests).var(axis=0) * np.array(rtime)
    plt.plot(var_per_recip_runtime)
plt.legend([name for name in results.keys()])
plt.xlabel('Level')
plt.ylabel('Variance per Reciprocal Runtime')
plt.yscale('log')

Variance per "reciprocal of the runtime" is considered because the varince is propotional to the  "reciprocal of the runtime". As we increase the computational complexity (runtime), the decrease in varince is inversely propotional to the complexity. 

## Cost comparison of gradients

In [ ]:
L = 13
objectives = {
    'NMC':      lambda x,y,beta0,beta,alpha,mu,sigma,level: IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=2**level),
    'MLMC':     lambda x,y,beta0,beta,alpha,mu,sigma,level: IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=level, w0=0.95, b=1.8, randomize=False),
    'RandMLMC': lambda x,y,beta0,beta,alpha,mu,sigma,level: IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=level, w0=0.95, b=1.8, randomize=True),
    'SUMO':     lambda x,y,beta0,beta,alpha,mu,sigma,level: IWELBO_SUMO(x, y, beta0, beta, alpha, mu, sigma, K_max=2**level)
}

def d(f):
    # return the derivative of f
    # returned value is a function
    def df(x,y,mu,sigma,level):
        param = tf.concat([beta, [beta0], [alpha]], axis=0)
        param = tf.Variable(param, dtype=tf.float64)
        with tf.GradientTape(persistent=True) as g:
            g.watch(param)
            beta_ = param[:D]
            beta0_ = param[D]
            alpha_ = param[D+1]
            target = f(x,y,beta0_,beta_,alpha_,mu,sigma,level)
        est = g.gradient(target, param)
        return est
    return df

results = {'NMC':[], 'MLMC':[], 'RandMLMC':[], 'SUMO':[]}
runtime = {'NMC':[], 'MLMC':[], 'RandMLMC':[], 'SUMO':[]}

for name, obj in objectives.items():
    
    # evaluate variance
    for i in range(100):
        results[name].append([])
        x,y,_ = generate_data(N=4000, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
        mu, sigma = laplace_approx(x, y, beta0, beta, alpha)
        for level in range(L):
            results[name][i].append( d(obj)(x,y,mu,sigma,level).numpy() )
    
    # evaluate runtime
    x,y,_ = generate_data(N=20000, D=3, T=2, beta0=beta0, beta=beta, alpha=alpha)
    mu, sigma = laplace_approx(x, y, beta0, beta, alpha)    
    for level in range(L):
        # Avoid the memery runout by 
        # manipulating the case of NMC with large n_MC (large level)
        if level>10 and name=='NMC':
            start = time.time()
            d(obj)(*[vec[:200] for vec in [x,y,mu,sigma]], level)
            end = time.time()
            runtime[name].append((end - start)*100)
        else:
            start = time.time()
            d(obj)(x,y,mu,sigma,level)
            end = time.time()
            runtime[name].append(end - start)

In [ ]:
for ests, rtime in zip(results.values(), runtime.values()):
    var_per_recip_runtime = np.array(ests).var(axis=0).sum(axis=1) * np.array(rtime)
    plt.plot(var_per_recip_runtime)
plt.legend([name for name in results.keys()])
plt.xlabel('Level')
plt.ylabel(r'Variance ( tr(Cov) ) per Reciprocal Runtime')
plt.yscale('log')

## Maximum Likelihood by Different Approximation

In [ ]:
def get_mlmc_cost(N, max_level, b, w0):
    # compute the cost of MLMC estimation 
    # when the size of x (and that of y) is N
    if max_level==0:
        levels = np.array([0])
        weights = np.array([1.])
    else:
        weights = 2.**(-(b+1)/2*np.arange(max_level))
        weights /= sum(weights)
        weights = np.concatenate([[w0], (1-w0)*weights])
        levels = np.arange(max_level+1)
    cost = np.ceil(N * weights[0])\
            + N * sum( np.ceil(weights[1:] * (2**levels[1:] + 2**(levels[1:]-1))) )
    return cost

In [ ]:
objectives = {
    "signorm":   lambda x, y, beta0, beta, alpha, mu, sigma: sigmoid_normal_likelihood(x, y, beta0, beta, alpha),
    "elbo":      lambda x, y, beta0, beta, alpha, mu, sigma: IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=1),
    "iwelbo8":   lambda x, y, beta0, beta, alpha, mu, sigma: IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=8),
    "iwelbo64":  lambda x, y, beta0, beta, alpha, mu, sigma: IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=64),
    "iwelbo512": lambda x, y, beta0, beta, alpha, mu, sigma: IWELBO(x, y, beta0, beta, alpha, mu, sigma, n_MC=512),
    "iwelbo512_mlmc": lambda x, y, beta0, beta, alpha, mu, sigma: IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=9, w0=0.90, b=1.8, randomize=False),
    "iwelbo512_randmlmc": lambda x, y, beta0, beta, alpha, mu, sigma: IWELBO_MLMC(x, y, beta0, beta, alpha, mu, sigma, max_level=9, w0=0.90, b=1.8, randomize=True),
    "iwelbo512_sumo": lambda x, y, beta0, beta, alpha, mu, sigma: IWELBO_SUMO(x, y, beta0, beta, alpha, mu, sigma, K_max=512),

}
N,T,D = (1000, 2, 3) if tf.test.is_gpu_available() else (200, 2, 3)

n_repeat = 10
params_repeated = {name:[] for name in objectives.keys()}

for name, obj in objectives.items():
    alpha_s = []
    beta0_s = []
    beta_s = []
    for i in range(n_repeat):
        print("training {}.... #iter:{} ".format(name,i))

        beta0_ = tf.Variable(0., dtype=tf.float64)
        beta_  = tf.Variable(np.zeros([D]), dtype=tf.float64)
        alpha_   = tf.Variable(1., dtype=tf.float64)

        # Gradient Descent
        for t in range(2001):

            rho_t = 0.5/(1+t)**0.7
            x,y,_ = generate_data(N, D, T, beta0, beta, alpha)
            # balance the cost of mlmc and nmc when level=9 (n_MC=512)
            if 'mlmc' in name:
                cost_nmc  = N * 2**9
                cost_mlmc = get_mlmc_cost(N, max_level=9, b=1.8, w0=0.9)
                N_mlmc = np.math.ceil(N * (cost_nmc / cost_mlmc))
                x,y,_ = generate_data(N_mlmc, D, T, beta0, beta, alpha)    

            with tf.GradientTape() as g:
                g.watch([beta0_, beta_, alpha_])
                mu, sigma = laplace_approx(x, y, beta0_.numpy(), beta_.numpy(), alpha_.numpy())
                score = obj(x, y, beta0_, beta_, alpha_, mu, sigma)
            dbeta0_, dbeta_, dalpha_ = g.gradient(score, [beta0_, beta_, alpha_])

            beta0_ = beta0_ + rho_t*dbeta0_
            beta_ = beta_ + rho_t*dbeta_
            alpha_ = alpha_ + dalpha_
            if t%200==0 and i==0:
                print("#iter: {},\tloss: {}".format(t, -score.numpy()))
        alpha_s.append(alpha_.numpy())
        beta0_s.append(beta0_.numpy())
        beta_s.append(beta_.numpy())
    print()
    params_repeated[name] = {
            'alpha': np.array(alpha_s),
            'beta0': np.array(beta0_s),
            'beta': np.array(beta_s)
    }

In [ ]:
def expand(key, val):
    # expand {"name":array([1,2,3,4,5])}
    # into {"name1":1, "name2":2, ..., "name5":5}
    if type(val)==np.ndarray:
        return {key+str(i+1): x for i,x in enumerate(val)} 
    else:
        return {key:val} 

def expand_param(param):
    expanded_param = {}
    for key, val in param.items():
        expanded_param.update(expand(key,val))
    return expanded_param

In [ ]:
params = {'ground_truth': expand_param(param0)}
params['ground_truth'].update({'MSE':0})
for name in objectives.keys():
    param_repeated = params_repeated[name]
    param_mean   = expand_param({name: array.mean(axis=0) for name, array in  param_repeated.items()})
    param_var = expand_param({name: array.std(axis=0) for name, array in  param_repeated.items()})
    param = {name_:'{:.5f} ± {:.5f}'.format(mean,var**(1/2.)) 
             for name_, mean, var 
             in zip( param_mean.keys(), param_mean.values(), param_var.values() )}
    error = [var+(mean-true_mean)**2 
             for var, mean, true_mean 
             in zip( param_var.values(), param_mean.values(), params['ground_truth'].values() )]
    MSE = sum(error)
    param.update({'MSE':MSE})
    params.update({name :param})

In [ ]:
data = pd.DataFrame(params).T
data

In [ ]:
data.to_csv('../out/random_effect_logistic_regression/MLE_error_{}.csv'.format(timestamp()))

#### Bottom Line: 
- IWELBO with large number of inner MC samples gives better estiamte than ELBO or sigmoid normal integral approximation, even for this simple model. 
- MLMC is more effective than nested MC when used for maximum likelihood estimation.